# Experiemet with Long Context Window Models

## Preparations

In [224]:
import os
from google import genai
from google.genai import types
from google.genai.types import UserContent, ModelContent, Part, FileData, Blob
import anthropic
from IPython.display import display, Image, Markdown
import PIL.Image
import requests
from io import BytesIO
from pdf2image import convert_from_path
from dotenv import load_dotenv
from IPython.display import Markdown
import base64

In [96]:
load_dotenv()

True

In [178]:
gemini_client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [98]:
anthropic_client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

In [99]:
# Function to load a PDF file and convert it to an image for Gemini
def load_pdf_for_llm(pdf_path):
    """
    Load a PDF file from local path and prepare it for Gemini Vision model.
    
    Args:
        pdf_path (str): Path to the local PDF file
        
    Returns:
        PIL.Image: The first page of the PDF as an image
    """
    try:    
        # Convert the first page of the PDF to an image
        # Convert all pages of the PDF to images
        images = convert_from_path(pdf_path)
        
        if images:
            return images
        else:
            print("No images extracted from the PDF.")
            return None
    except Exception as e:
        print(f"Error loading PDF: {e}")
        return None

In [134]:
def load_image_for_anthropic(images):
    image_content = []
    for i, image in enumerate(images):
        buffered = BytesIO()
        image.save(buffered, format = "JPEG")
        encoded_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
        image_content.append(
            {
                "type": "text",
                "text": f"Image {i+1}"
            }
        )
        image_content.append(
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": encoded_image
                }
            }
        )
    return image_content

In [116]:
pdf_dir = "pdf_collection"
pdf_files =sorted(os.listdir(pdf_dir))
pdf_files

['1_empty_graph.pdf',
 '2_screenshot_text_and_image.pdf',
 '3_complex_graph.pdf',
 '4_syllabus.pdf',
 '5_table.pdf',
 '6_apple_10k_long_documents.pdf',
 'hand_written_math.pdf',
 'long_lecture_slide.pdf']

In [105]:
files = []
for pdf_file in pdf_files:
    print("Processing PDF file:", pdf_file)
    images = load_pdf_for_llm(os.path.join(pdf_dir, pdf_file))
    if images:
        print("Number of images extracted:", len(images))
    else:
        print("No images extracted from the PDF.")
    files.append(images)

Processing PDF file: 1_empty_graph.pdf
Number of images extracted: 1
Processing PDF file: 2_screenshot_text_and_image.pdf


python(83235) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83236) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83237) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83238) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83239) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83240) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Number of images extracted: 1
Processing PDF file: 3_complex_graph.pdf
Number of images extracted: 1
Processing PDF file: 4_syllabus.pdf


python(83241) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83242) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83243) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83244) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83245) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83246) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Number of images extracted: 9
Processing PDF file: 5_table.pdf
Number of images extracted: 1
Processing PDF file: 6_apple_10k_long_documents.pdf


...

Number of images extracted: 121
Processing PDF file: hand_written_math.pdf


python(83258) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83259) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83261) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Number of images extracted: 7
Processing PDF file: long_lecture_slide.pdf


python(83264) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83265) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(83266) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Number of images extracted: 68


In [51]:
type(files[1][0])

PIL.PpmImagePlugin.PpmImageFile

## Experiments

### Experiment 1: Empty_Graph

#### Gemini-2.5-pro-exp-03-25

In [124]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["tell me about this graph?", files[0]])
from IPython.display import Markdown
display(Markdown(response.text))

Okay, let's break down this graph:

1.  **Type of Graph:** This is a **pie chart**. Pie charts are used to show proportions of a whole. The entire circle represents 100% of something.

2.  **Sections/Slices:** The chart is divided into **three sections** (or slices), each represented by a different color:
    *   Blue
    *   Red (or reddish-orange)
    *   Yellow (or gold)

3.  **Proportions (Visual Estimation):**
    *   The **blue section is the largest**, appearing to represent slightly more than half (maybe around 55-60%) of the total.
    *   The **yellow section is the next largest**, looking like roughly a quarter (perhaps 20-25%) of the total.
    *   The **red section is the smallest**, appearing slightly smaller than the yellow section (maybe 15-20%).

4.  **Missing Information:**
    *   **Labels/Legend:** There are no labels or a legend to tell us what each colored section represents (e.g., categories, groups, survey responses).
    *   **Values:** No specific percentages or numerical values are shown for each slice.
    *   **Title:** There is no title to explain the overall subject of the chart.

**In summary:** This is a simple pie chart showing a whole divided into three parts of different sizes. The blue part is the majority, followed by yellow, and then red. Without labels or values, we can only understand the relative proportions visually, not the specific data or context it represents.

#### Gemini-2.0-flash

In [125]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["tell me about this graph?", files[0]])
from IPython.display import Markdown
display(Markdown(response.text))

The image is a pie chart. It is divided into three segments: blue, red, and yellow. The blue segment takes up the largest portion of the pie, followed by red, and then yellow. Without numerical labels or percentages, it's impossible to determine the exact proportions of each segment, but we can visually estimate that blue is probably around 50%, red is around 25-30%, and yellow is around 20-25%.

#### Claude 3.5 Haiku

In [143]:
image_content = load_image_for_anthropic(files[0])
image_content.append({"type": "text", "text": "tell me about this graph?"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

This image is a pie chart with four sections in different colors:

1. Blue (largest section): Appears to cover approximately 50-60% of the total pie chart area
2. Red (medium section): Covers roughly 20-25% of the chart
3. Yellow (smaller section): Takes up about 10-15% of the chart
4. Blue (smallest section): A small wedge at the top of the chart

The colors are bright and solid, and the chart has a clean, simple design. Without additional context, I can't specify what data these sections represent. Pie charts are typically used to show proportions or percentages of a whole, with each slice representing a different category or segment.

Would you like me to elaborate on any aspect of the chart?

### Experiment 2: Screenshot_Text_and_Image

#### Gemini-2.5-pro-exp-03-25

In [29]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["Describe this content?", files[1]])
from IPython.display import Markdown
display(Markdown(response.text))

This image is a **promotional graphic or advertisement banner**.

Here's a breakdown of its content:

1.  **Headline:** "Sample our Education Journals" - Clearly states the subject matter being promoted.
2.  **Call to Action:** ">> Sign in here to start your access to the latest two volumes for 14 days".
    *   "Sign in here" is highlighted (likely a hyperlink in a digital context).
    *   It specifies the benefit: access to the "latest two volumes".
    *   It details the duration of the offer: "for 14 days".
3.  **Visual Elements:**
    *   A solid blue background.
    *   White text for readability.
    *   A white icon in the top right corner, which appears to be a stylized oil lamp or "lamp of knowledge," commonly associated with education and learning.

In summary, it's an **advertisement offering a 14-day free trial access to recent education journals**, requiring users to sign in to activate the trial.

#### Gemini-2.0-flash

In [35]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["Describe this content?", files[1]])
from IPython.display import Markdown
display(Markdown(response.text))

The image is an advertisement on a blue background. The text on the image reads "Sample our Education Journals." Below that, it says ">> Sign in here to start your access to the latest two volumes for 14 days". There is also a small icon of an oil lamp on the top right of the image.

#### Claude 3.5 Haiku

In [144]:
image_content = load_image_for_anthropic(files[1])
image_content.append({"type": "text", "text": "Describe this content?"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

This image is a sign-in page or banner for "Sample our Education Journals". The banner is in a blue color scheme with white text. It invites users to "Sign In here to start your access to the latest two volumes for 14 days". There's a small white icon of what appears to be a bowl or cup in the top right corner of the banner. The text suggests this is a trial or preview access to educational journal volumes, allowing users 14 days of complimentary access after signing in.

### Experiment 3: Complex_Graph

#### Gemini-2.5-pro-exp-03-25

In [146]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["What's inside Figure 2?", files[2]])
from IPython.display import Markdown
display(Markdown(response.text))

Figure 2 illustrates the architecture of the **Pixtral Vision Encoder**.

Here's a breakdown of the components shown:

1.  **Image Patches:** The input starts with image patches from potentially multiple images with variable sizes and aspect ratios.
2.  **RoPE-2D:** Rotary Position Embeddings (2D) are applied to the patches to handle variable image sizes.
3.  **Block-diagonal attention mask:** This mask is applied to enable sequence packing, allowing efficient batching of images/patches while preventing attention leakage between different images in the batch.
4.  **Pixtral-ViT:** The core Vision Transformer component, specifically adapted for the Pixtral model (indicated as a Bidirectional transformer).
5.  **Vision-Language Projector:** A layer that likely maps the visual features extracted by the ViT into a space compatible with a language model.
6.  **MLP:** A Multi-Layer Perceptron for further processing.
7.  **Output Embeddings:** The final output representation of the processed image patches.

The diagram also visually represents how sequence packing works using block-diagonal attention and includes special tokens like `[IMG_BREAK]` and `[IMG_END]`. The overall purpose is to show how Pixtral encodes images, particularly focusing on its ability to handle variable input sizes and efficient batching.

#### Gemini-2.0-flash

In [36]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What's inside Figure 2?", files[2]])
from IPython.display import Markdown
display(Markdown(response.text))

Figure 2 illustrates the Pixtral Vision Encoder. It shows the flow of information through the encoder, starting with Image Patches, then RoPE-2D (position encodings), a Block-diagonal attention mask, a Bidirectional Transformer, Pixtral-ViT, a Vision-Language Projector, an MLP, and finally, Output Embeddings. The figure also shows that "Add [IMG_BREAK] and [IMG_END] tokens" is performed after the Vision-Language Projector.


#### Claude 3.5 Haiku

In [145]:
image_content = load_image_for_anthropic(files[2])
image_content.append({"type": "text", "text": "What's inside Figure 2?"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

Figure 2 shows the architecture of the Pixtral Vision Encoder. It's a detailed diagram that illustrates the components and data flow of the vision encoding process. The figure includes:

1. A block diagram with multiple layers, including:
- Output Encoding
- Vision-Language Projector
- Pixtral-VIT (Vision Transformer)

2. Color-coded blocks representing different stages of image processing

3. Small visual representations on the right side, including:
- A pie chart
- A histogram or heatmap-like image

The caption explains that Pixtral uses a new vision encoder trained from scratch to natively support variable image sizes and aspect ratios. It highlights features like block-diagonal attention masks and sequence packing for batching, and notes that the encoder facilitates variable image sizes while supporting 2D encodings.

The diagram provides a visual breakdown of the technical architecture and data transformation process of the Pixtral Vision Encoder.

### Experiment 4: Syllabus

#### Gemini-2.5-pro-exp-03-25

In [147]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["What's covered in week 2?", files[3]])
from IPython.display import Markdown
display(Markdown(response.text))

According to the syllabus provided (page 2, under "Weekly Structure"), **Week 2** is part of the block covering **SQL Language**.

This block spans Weeks 2-4 and includes the following topics (as detailed on page 3):
*   Introduction to SQL
*   Intermediate SQL
*   Advanced SQL

Therefore, Week 2 will focus on the initial parts of learning SQL Language, likely starting with the "Introduction to SQL".

#### Gemini-2.0-flash

In [37]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What's covered in week 2?", files[3]])
from IPython.display import Markdown
display(Markdown(response.text))

Weeks 2-4 covers SQL language, including:

*   Introduction to SQL
*   Intermediate SQL
*   Advanced SQL

#### Claude 3.5 Haiku

In [148]:
image_content = load_image_for_anthropic(files[3])
image_content.append({"type": "text", "text": "What's covered in week 2?"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

According to the course syllabus (Image 3), in Weeks 2-4, the course will cover SQL Language. More specifically, the topics for SQL Language include:

- Introduction to SQL
- Intermediate SQL
- Advanced SQL

### Experiment 5: Table

#### Gemini-2.5-pro-exp-03-25

In [149]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["Extract the table from this image and present it in a table format", files[4]])
from IPython.display import Markdown
display(Markdown(response.text))

Okay, here is the table extracted from the image:

| Category/Label                    | Percentage % | Count |
| :-------------------------------- | :----------- | :---- |
| **Gender**                        |              |       |
| Female                            | 49.0%        | 540   |
| Male                              | 51.0%        | 561   |
| Early Childhood                   | 3.3%         | 36    |
| **What grades do you teach in school?** |              |       |
| Elementary School                 | 21.0%        | 231   |
| High School                       | 29.4%        | 324   |
| Primary School                    | 46.3%        | 510   |
| **Educational Levels**            |              |       |
| Bachelor                          | 83.1%        | 915   |
| Master                            | 12.0%        | 132   |
| PhD                               | 2.7%         | 30    |
| Diploma                           | 2.2%         | 24    |
| **Teaching Experience**           |              |       |
| 0–5                               | 24.3%        | 267   |
| 11–15                             | 36.8%        | 405   |
| 6–10                              | 37.6%        | 414   |
| +16                               | 1.4%         | 15    |

**Note:** The item "Early Childhood" appears structurally under "Gender" in the original image, but contextually might relate more to the "Grades Taught" section. Similarly, "Bachelor" appears before the "Educational Levels" heading but clearly belongs to that category. The table above reflects the data points and structure as presented, grouping "Bachelor" under "Educational Levels" for clarity.

#### Gemini-2.0-flash

In [150]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["Extract the table from this image and present it in a table format", files[4]])
from IPython.display import Markdown
display(Markdown(response.text))

Here is the table extracted from the image:

| Category                    |   Percentage % |   Count |
|:----------------------------|---------------:|--------:|
| **Gender**                  |                |         |
| Female                      |           49   |     540 |
| Male                        |           51   |     561 |
| Early Childhood             |            3.3 |      36 |
| **What grades do you teach in school?**       |                |         |
| Elementary School           |           21   |     231 |
| High School                 |           29.4 |     324 |
| Primary School              |           46.3 |     510 |
| Bachelor                    |           83.1 |     915 |
| **Educational Levels**      |                |         |
| Master                      |           12   |     132 |
| PhD                         |            2.7 |      30 |
| Diploma                     |            2.2 |      24 |
| **Teaching Experience**     |                |         |
| 0-5                         |           24.3 |     267 |
| 11-15                       |           36.8 |     405 |
| 6-10                        |           37.6 |     414 |
| +16                         |            1.4 |      15 |

#### Claude 3.5 Haiku

In [151]:
image_content = load_image_for_anthropic(files[4])
image_content.append({"type": "text", "text": "Extract the table from this image and present it in a table format"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

Here's the table extracted from the image:

| Category | Percentage % | Count |
|---|---|---|
| **Gender** |  |  |
| Female | 49.0% | 540 |
| Male | 51.0% | 561 |
| Early Childhood | 3.3% | 36 |
| **What grades do you teach in school?** |  |  |
| Elementary School | 21.0% | 231 |
| High School | 29.4% | 324 |
| Primary School | 46.3% | 510 |
| **Educational Levels** |  |  |
| Bachelor | 83.1% | 915 |
| Master | 12.0% | 132 |
| PhD | 2.7% | 30 |
| Diploma | 2.2% | 24 |
| **Teaching Experience** |  |  |
| 0-5 | 24.3% | 267 |
| 11-15 | 36.8% | 405 |
| 6-10 | 37.6% | 414 |
| +16 | 1.4% | 15 |

### Experiment 6: Apple 10K - Long Document

#### Gemini-2.5-pro-exp-03-25

In [155]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["What are the product, service and software annoucnements?", files[5]])
from IPython.display import Markdown
display(Markdown(response.text))

Based on the provided Apple Inc. Form 10-K for the fiscal year ended September 28, 2024, the product, service, and software announcements during that year are listed on page 21 under "Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations".

Here are the significant announcements mentioned for fiscal year 2024:

**First Quarter 2024:**

*   MacBook Pro 14-in.
*   MacBook Pro 16-in.
*   iMac

**Second Quarter 2024:**

*   MacBook Air 13-in.
*   MacBook Air 15-in.

**Third Quarter 2024:**

*   iPad Air
*   iPad Pro
*   iOS 18, macOS Sequoia, iPadOS 18, watchOS 11, visionOS 2 and tvOS 18 (updates to the Company’s operating systems)
*   Apple Intelligence™ (a personal intelligence system that uses generative models)

**Fourth Quarter 2024:**

*   iPhone 16, iPhone 16 Plus, iPhone 16 Pro and iPhone 16 Pro Max
*   Apple Watch Series 10
*   AirPods 4

In [156]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["What's the segment operating performance for 2024, by geography? Provide the answer in JSON with two keys: answer and page_number}", files[5]])
from IPython.display import Markdown
display(Markdown(response.text))

```json
{
  "answer": {
    "Americas": {
      "Net sales (millions)": 167045,
      "Operating income (millions)": 67656
    },
    "Europe": {
      "Net sales (millions)": 101328,
      "Operating income (millions)": 41790
    },
    "Greater China": {
      "Net sales (millions)": 66952,
      "Operating income (millions)": 27082
    },
    "Japan": {
      "Net sales (millions)": 25052,
      "Operating income (millions)": 12454
    },
    "Rest of Asia Pacific": {
      "Net sales (millions)": 30658,
      "Operating income (millions)": 13062
    }
  },
  "page_number": 47
}
```

In [157]:
response = gemini_client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=["How has products and servcies revenue changed from 2023 to 2024? Provide the answer in JSON with two keys: answer and page_number}", files[5]])
display(Markdown(response.text))

```json
{
  "answer": "From 2023 to 2024, Products net sales decreased from $298,085 million to $294,866 million. Services net sales increased from $85,200 million in 2023 to $96,169 million in 2024.",
  "page_number": 23
}
```

#### Gemini-2.0-flash

In [ ]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What are the product, service and software annoucnements?", files[5]])
display(Markdown(response.text))

Okay, here are the product, service, and software announcements made by Apple during the fiscal year of 2024 according to the 10-K report:

**First Quarter 2024:**

*   MacBook Pro 14-in.
*   MacBook Pro 16-in.
*   iMac

**Second Quarter 2024:**

*   MacBook Air 13-in.
*   MacBook Air 15-in.

**Third Quarter 2024:**

*   iPad
*   iPad Pro
*   iOS 18, macOS Sequoia, iPadOS 18, watchOS 11, visionOS 2 and tvOS 18, updates to the Company’s operating systems
*   Apple Intelligence™, a personal intelligence system that uses generative models

**Fourth Quarter 2024:**

*   iPhone 16, iPhone 16 Plus, iPhone 16 Pro and iPhone 16 Pro Max
*   Apple Watch Series 10
*   AirPods 4



In [153]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What's the segment operating performance for 2024, by geography? Provide the answer in JSON with two keys: answer and page_number}", files[5]])
display(Markdown(response.text))

```json
{
  "answer": "{\n \"Americas\": {\n \"Net sales\": \"167,045\",\n \"Operating income\": \"67,656\"\n },\n \"Europe\": {\n \"Net sales\": \"101,328\",\n \"Operating income\": \"41,790\"\n },\n \"Greater China\": {\n \"Net sales\": \"66,862\",\n \"Operating income\": \"27,002\"\n },\n \"Japan\": {\n \"Net sales\": \"25,652\",\n \"Operating income\": \"12,454\"\n },\n \"Rest of Asia Pacific\": {\n \"Net sales\": \"30,658\",\n \"Operating income\": \"13,052\"\n }\n}",
  "page_number": 47
}
```

In [ ]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["How has products and servcies revenue changed from 2023 to 2024? Provide the answer in JSON with two keys: answer and page_number}", files[5]])
display(Markdown(response.text))

```json
{
  "answer": "Services net sales increased during 2024 compared to 2023 due primarily to higher net sales from advertising, the App Store, and cloud services.",
  "page_number": 23
}
```

#### Claude 3.5 Haiku (Aborted due to context window limit)

In [ ]:
image_content = load_image_for_anthropic(files[5])
image_content.append({"type": "text", "text": "What are the product, service and software annoucnements?"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

In [ ]:
image_content.append({"type": "text", "text": "What's the segment operating performance for 2024, by geography? Provide the answer in JSON with two keys: answer and page_number}"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

In [ ]:
image_content.append({"type": "text", "text": "What are the product, service and software annoucnements?"})

response = anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022", 
    messages=[
        {
            "role": "user",
            "content": image_content
        }
    ],
    max_tokens=4096
)
display(Markdown(response.content[0].text))

## Gemini Caching

In [ ]:
cache = gemini_client.caches.create(
    model = "gemini-2.0-flash",
    config = types.CreateCachedContentConfig(
        display_name = "apple_10k_cache",
        system_instruction = (
            "You are a financial analyst that can answer questions about the Apple 10K report."
        ),
        contents = [files[2]],
        ttl = "600s"
    ),
)

In [71]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What are the product, service and software annoucnements?"],
    config = types.GenerateContentConfig(
        cached_content = cache.name,
        temperature = 0.0,
    )
)
from IPython.display import Markdown
display(Markdown(response.text))

Here are the product, service, and software announcements from the 10K report:

**First Quarter 2024:**

*   MacBook Pro 14-in.
*   MacBook Pro 16-in.
*   iMac

**Second Quarter 2024:**

*   MacBook Air 13-in.
*   MacBook Air 15-in.

**Third Quarter 2024:**

*   iPad
*   iPad Pro
*   iOS 18, macOS Sequoia, iPadOS 18, watchOS 11, visionOS 2 and tvOS 18, updates to the Company's operating systems
*   Apple Intelligence, a personal intelligence system that uses generative models

**Fourth Quarter 2024:**

*   iPhone 16, iPhone 16 Plus, iPhone 16 Pro and iPhone 16 Pro Max
*   Apple Watch Series 10
*   AirPods 4

In [ ]:
response = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["What's the segment operating performance for 2024, by geography? Provide the answer in JSON with two keys: answer and page_number}"],
    config = types.GenerateContentConfig(
        cached_content=cache.name,
        temperature = 0.0,
    )
)
from IPython.display import Markdown
display(Markdown(response.text))

```json
{
"answer": "Americas: $167,045\nEurope: $101,328\nGreater China: $66,652\nJapan: $25,652\nRest of Asia Pacific: $30,658\nTotal net sales: $391,335",
"page_number": 22
}
```

In [ ]:
response = gemini_client.models.generate_content(
    model = "gemini-2.0-flash",
    contents = ["How has products and servcies revenue changed from 2023 to 2024? Provide the answer in JSON with two keys: answer and page_number}"],
    config = types.GenerateContentConfig(
        cached_content = cache.name
    )
)
display(Markdown(response.text))

```json
{
  "answer": "From 2023 to 2024, iPhone revenue decreased from $200,563 million to $201,183 million. Mac revenue increased from $29,350 million to $29,984 million. iPad revenue decreased from $26,850 million to $25,694 million. Wearables, Home and Accessories revenue decreased from $39,840 million to $37,005 million. Services revenue increased from $85,250 million to $96,169 million.",
  "page_number": 23
}
```

### Observations
- Caching itself takes quite some time
- Cashing costs: 
    - $1.00/1,000,000 tokens per hour
    - the cached tokens are then billed at $0.025/1,000,000 tokens, 1/4 of the original cost ($0.1/1,000,000 tokens) when used in following prompts
- The minimum input token count for context caching is 4,096, and the maximum is the same as the maximum for the given model
- For cashed documents, the reponse time is roughly 1/4 the time of the non-cached responses

### Quick Cost Estimate
- Assumptions: 30 page lecture slides, 60,000 tokens, 30 minutes study session, 20 questions
- No-Caching Cost: 60,000 * 20 / 1,000,000 * 0.1 = $0.12
- Caching Cost: 60,000 / 1,000,000 * 0.5 * 1 + 60,000 * 20 / 1,000,000 * 0.025 = $0.06
- Intuition: longer caching time makes caching less cost-effective, more questions asked makes caching more cost-effective

## Message Chaining

#### Older API

In [172]:
import google.generativeai as genai_sdk

In [173]:
genai_sdk.configure(api_key=os.environ["GEMINI_API_KEY"])
gemini_model = genai_sdk.GenerativeModel("gemini-2.0-flash")

In [174]:
chat = gemini_model.start_chat()

In [175]:
response1 = chat.send_message("Who is Elon Musk?")
print(display(Markdown(response1.text)))

response2 = chat.send_message("When was he born?")
print(display(Markdown(response2.text)))

Elon Musk is a South African-born American entrepreneur and businessman. He is best known as the founder, CEO, and CTO of SpaceX; early-stage investor, CEO, and product architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI.

Here's a breakdown of his key roles and accomplishments:

*   **SpaceX:** Musk founded SpaceX with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX has achieved numerous milestones, including being the first private company to successfully launch, orbit, and recover a spacecraft, and to send humans to the International Space Station.

*   **Tesla:** Musk is the CEO and product architect of Tesla, a company that designs, manufactures, and sells electric vehicles, battery energy storage systems, and solar products. Tesla has played a significant role in the popularization of electric vehicles and the advancement of battery technology.

*   **The Boring Company:** Musk founded The Boring Company to address traffic congestion by building underground tunnels.

*   **Neuralink:** Musk co-founded Neuralink, a neurotechnology company developing implantable brain-machine interfaces.

*   **OpenAI:** Musk co-founded OpenAI, a non-profit artificial intelligence research company with the goal of promoting and developing friendly AI in a way that benefits humanity.

Musk is a highly influential figure in the technology industry and is known for his ambitious and sometimes controversial ideas. He is a frequent commentator on social media and often shares his views on a wide range of topics, from artificial intelligence to climate change.

None


Elon Musk was born on **June 28, 1971**.


None


#### New API

In [216]:
buffered = BytesIO()
files[0][0].save(buffered, format = "JPEG")
encoded_image = base64.b64encode(buffered.getvalue()).decode('utf-8')

In [210]:
chat_session = gemini_client.chats.create(
    model="gemini-2.0-flash",
    history=[],
    # config=types.GenerateContentConfig(
    #     cached_content="apple_10k_cache",
    #     temperature=0.0,
    # )
)


In [227]:
response = chat_session.send_message([
    Part(text="Describe this image?"),
    Part(inline_data=Blob(
        mime_type="image/jpeg",
        data=encoded_image
    ))
])

display(Markdown(response.text))

The image is a pie chart with three slices. The largest slice, approximately 50% of the chart, is blue. A smaller slice, roughly 25% of the chart, is red. The remaining slice, also around 25% of the chart, is yellow. The pie chart is set against a plain white background.

In [226]:
list(chat_session.get_history())

[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Describe this image?'), Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=Blob(data=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\x08\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x06u\t#\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x0